# ***Divvy Trips 2019 Analysis***


This analysis is based on the Divvy case study "'Sophisticated, Clear, and Polished': Divvy and Data Visualization" written by Kevin Hartman (found here: https://artscience.blog/home/divvy-dataviz-case-study). The purpose of this script is to consolidate downloaded Divvy data into a single dataframe and then conduct simple analysis to help answer the key question: "In what ways do members and casual riders use Divvy bikes differently?"

*Installing the necessary packages*

In [62]:
library(tidyverse)

library(lubridate)

library(ggplot2)

library(dplyr)

In [67]:
help(tidyverse)

* library(tidyverse) #helps wrangle data 
 
* library(lubridate) #helps wrangle date attributes
 
* library(ggplot2) #helps visualize data

* getwd() #displays your working directory 

* setwd("/Users/kevinhartman/Desktop/Divvy_Exercise/csv") #sets your working directory to simplify calls to data … make sure to use your OWN username instead of mine ;)

# STEP 1: COLLECT DATA

In [5]:
a = read_csv("../input/divvy-trips/Divvy_Trips_2019_Q2.csv")

b = read_csv("../input/divvy-trips/Divvy_Trips_2019_Q3.csv")

c = read_csv("../input/divvy-trips/Divvy_Trips_2019_Q4.csv")

d = read_csv("../input/divvy-trips/Divvy_Trips_2020_Q1.csv")


In [ ]:
#str(a)

In [ ]:
#str(a)

In [6]:
colnames(a)

In [7]:
colnames(b)

In [8]:
colnames(c)

In [9]:
colnames(d)

# STEP 2: WRANGLE DATA AND COMBINE INTO A SINGLE FILE 

Compare column names each of the files 

#### While the names don't have to be in the same order,

they DO need to match perfectly before we can use a command to join them into one file 

In [10]:
a <- rename(a ,ride_id = "01 - Rental Details Rental ID" ,rideable_type = "01 - Rental Details Bike ID" ,started_at = "01 - Rental Details Local Start Time",ended_at = "01 - Rental Details Local End Time",start_station_name = "03 - Rental Start Station Name" ,start_station_id = "03 - Rental Start Station ID" ,end_station_name = "02 - Rental End Station Name" ,end_station_id = "02 - Rental End Station ID" ,member_casual = "User Type")

b <- rename(b,ride_id = trip_id ,rideable_type = bikeid ,started_at = start_time,ended_at = end_time,start_station_name = from_station_name ,start_station_id = from_station_id ,end_station_name = to_station_name ,end_station_id = to_station_id ,member_casual = usertype)

c <- rename(c ,ride_id = trip_id ,rideable_type = bikeid ,started_at = start_time,ended_at = end_time,start_station_name = from_station_name ,start_station_id = from_station_id ,end_station_name = to_station_name ,end_station_id = to_station_id ,member_casual = usertype)


**Inspect the dataframes and look for incongruencies**

In [11]:
str(a)
str(b)
str(c)
str(d)

In [12]:
head(a)

**Convert ride_id and rideable_type to character so that they can stack correctly**

In [20]:
a <- mutate(a, ride_id = as.character(ride_id) ,rideable_type = as.character(rideable_type))
b <- mutate(b, ride_id = as.character(ride_id) ,rideable_type = as.character(rideable_type))
c <- mutate(c, ride_id = as.character(ride_id) ,rideable_type = as.character(rideable_type), started_at= as.POSIXct(started_at, format = "%Y-%m-%dT%H:%M:%S+000") , ended_at= as.POSIXct(ended_at, format = "%Y-%m-%dT%H:%M:%S+000") ) 


**Stack individual quarter's data frames into one big data frame**

In [21]:
all_trips <- bind_rows(a,b,c,d)

**Remove lat, long, birthyear, and gender fields as this data was dropped beginning in 2020**

In [22]:
all_trips <- all_trips %>%
            select( -c ( start_lat, start_lng, end_lat, end_lng, birthyear, gender, "01 - Rental Details Duration In Seconds Uncapped", "05 - Member Details Member Birthday Year", "Member Gender", "tripduration"))

In [23]:
View(all_trips)

# STEP 3: CLEAN UP AND ADD DATA TO PREPARE FOR ANALYSIS

 Inspect the new table that has been created 
 
 * colnames(all_trips) #List of column names

* nrow(all_trips) #How many rows are in data frame?

* dim(all_trips) #Dimensions of the data frame?

* head(all_trips) #See the first 6 rows of data frame.

* tail(all_trips) str(all_trips) #See list of columns and data types (numeric, character, etc)

* summary(all_trips) #Statistical summary of data. Mainly for numerics

In [25]:
colnames(all_trips)

In [26]:
nrow(all_trips) 

In [29]:
dim(all_trips) 

In [31]:
head(all_trips)

In [32]:
tail(all_trips)

In [33]:
summary(all_trips) 

In [34]:
str(all_trips)

There are a few problems we will need to fix:

(1) In the "member_casual" column, there are two names for members ("member" and "Subscriber") and two names for casual riders ("Customer" and "casual"). We will need to consolidate that from four to two labels.

(2) The data can only be aggregated at the ride-level, which is too granular. We will want to add some additional columns of data – such as day, month, year – that provide additional opportunities to aggregate the data.

(3) We will want to add a calculated field for length of ride since the 2020Q1 data did not have the "tripduration" column. We will add "ride_length" to the entire dataframe for consistency.

(4) There are some rides where tripduration shows up as negative, including several hundred rides where Divvy took bikes out of circulation for Quality Control reasons. We will want to delete these rides.

In the "member_casual" column, replace "Subscriber" with "member" and "Customer" with "casual"

Before 2020, Divvy used different labels for these two types of riders … we will want to make our dataframe consistent with their current nomenclature

N.B.: "Level" is a special property of a column that is retained even if a subset does not contain any values from a specific level

Begin by seeing how many observations fall under each usertype





In [35]:
table(all_trips$member_casual)

**Reassign to the desired values (we will go with the current 2020 labels)**

In [36]:
all_trips <- all_trips %>% mutate(member_casual = recode(member_casual ,"Subscriber" = "member" ,"Customer" = "casual"))

**Check to make sure the proper number of observations were reassigned**

In [37]:

table(all_trips$member_casual)

**Add columns that list the date, month, day, and year of each ride . This will allow us to aggregate ride data for each month, day, or year … before completing these operations we could only aggregate at the ride level**

In [38]:
all_trips$date <- as.Date(all_trips$started_at) #The default format is yyyy-mm-dd

all_trips$month <- format(as.Date(all_trips$date), "%m") 

all_trips$day <- format(as.Date(all_trips$date), "%d")

all_trips$year <- format(as.Date(all_trips$date), "%Y") 

all_trips$day_of_week <- format(as.Date(all_trips$date), "%A")

**Add a "ride_length" calculation to all_trips (in seconds)**

In [41]:
all_trips $ ride_length <- difftime (all_trips $ ended_at , all_trips $ started_at)

**Inspect the structure of the columns**

In [42]:
str(all_trips)

**Convert "ride_length" from Factor to numeric so we can run calculations on the data**

In [44]:

is.factor(all_trips$ride_length) 



In [45]:
all_trips $ ride_length <- as.numeric(as.character(all_trips$ride_length)) 

is.numeric(all_trips$ride_length)

Remove "bad" data

The dataframe includes a few hundred entries when bikes were taken out of docks and checked for quality by Divvy or ride_length was negative

We will create a new version of the dataframe (v2) since data is being removed

In [71]:
all_trips_v2 <- all_trips [ ! ( all_trips $ start_station_name == "HQ QR" | all_trips $ ride_length<0) ,]

In [72]:
head(all_trips_v2)

# STEP 4: CONDUCT DESCRIPTIVE ANALYSIS

In [73]:
mean(all_trips_v2$ride_length) #straight average (total ride length / rides)


In [74]:
median(all_trips_v2$ride_length) #midpoint number in the ascending array of ride lengths

In [75]:

max(all_trips_v2$ride_length) #longest ride

In [76]:
 min(all_trips_v2$ride_length) # shortest ride

**You can condense the four lines above to one line using summary() on the specific attribute**

In [77]:
summary(all_trips_v2$ride_length)

**Compare members and casual users**

In [78]:
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = mean) 

aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = median) 

aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = max) 

aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = min)

**See the average ride time by each day for members vs casual users**

In [79]:
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN = mean)

**Notice that the days of the week are out of order. Let's fix that.**

In [80]:
all_trips_v2$day_of_week <- ordered(all_trips_v2$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

**Now, let's run the average ride time by each day for members vs casual users**

In [81]:
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN = mean)

**analyze ridership data by type and weekday**

In [83]:
all_trips_v2 %>% 
    mutate(weekday = wday(started_at, label = TRUE)) %>%  #creates weekday field using wday()
    group_by(member_casual, weekday) %>% #groups by usertype and weekday 
    summarise(number_of_rides = n() , average_duration = mean(ride_length)) %>% #calculates the number of rides and average duration & calculates the average duration
    arrange(member_casual, weekday) # sorts


**Let's visualize the number of rides by rider type**

In [84]:
all_trips_v2 %>% mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% summarise(number_of_rides = n() ,average_duration =     mean(ride_length)) %>% 
  arrange(member_casual, weekday) %>% 
  ggplot(aes(x = weekday, y = number_of_rides, fill = member_casual)) + geom_col(position =   "dodge")

**Let's create a visualization for average duration**

In [86]:
all_trips_v2 %>% mutate(weekday = wday(started_at, label = TRUE)) %>% 
    group_by(member_casual, weekday) %>%
    summarise(number_of_rides = n() ,average_duration = mean(ride_length)) %>% 
    arrange(member_casual, weekday) %>% 
    ggplot(aes(x = weekday, y = average_duration, fill = member_casual)) + geom_col (position = "dodge")

# STEP 5: EXPORT SUMMARY FILE FOR FURTHER ANALYSIS

Create a csv file that we will visualize in Excel, Tableau, or my presentation software # N.B.: This file location is for a Mac. If you are working on a PC, change the file location accordingly (most likely "C:\Users\YOUR_USERNAME\Desktop…") to export the data. You can read more here: 

In [87]:
counts <- aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + all_trips_v2$day_of_week, FUN = mean)

In [88]:

write.csv(counts, file = "C:\\Users\\HP\\Downloads\\...........\avg_ride_length.csv")